# Pré-processamento sequencial
Fluxo usa imagens reais da pasta `pdi/` para aplicar conversão para tons de cinza, filtros de suavização e segmentação, salvando cada etapa em `processed/`.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops, hog


In [ ]:
IMAGE_PATH = Path("apple.jpg")
OUTPUT_DIR = Path("processed")

if not IMAGE_PATH.exists():
    raise FileNotFoundError(
        f"Imagem {IMAGE_PATH} não encontrada. Coloque o arquivo na pasta 'pdi/'."
    )

input_bgr = cv2.imread(str(IMAGE_PATH))
if input_bgr is None:
    raise ValueError(f"Falha ao ler a imagem {IMAGE_PATH} com o OpenCV.")

image_label = IMAGE_PATH.stem
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_original.png"), input_bgr)

# 1. Escala de cinza
gray = cv2.cvtColor(input_bgr, cv2.COLOR_BGR2GRAY)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_gray.png"), gray)

# 2. Suavização (Gaussian)
gaussian = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_gaussian_blur.png"), gaussian)

# 3. Suavização (Mediana)
median = cv2.medianBlur(gray, 5)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_median_blur.png"), median)

# 4. Limiarização (Otsu)
_, otsu_mask = cv2.threshold(gaussian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_otsu_mask.png"), otsu_mask)

# 5. Detector de bordas (Canny)
edges = cv2.Canny(gaussian, 100, 200)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_canny_edges.png"), edges)


In [ ]:
contours, _ = cv2.findContours(otsu_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_overlay = input_bgr.copy()
cv2.drawContours(contour_overlay, contours, -1, (0, 255, 0), 2)
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_contours.png"), contour_overlay)

shape_features = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    x, y, w, h = cv2.boundingRect(cnt)
    circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0.0
    shape_features.append(
        {
            "area": float(area),
            "perimeter": float(perimeter),
            "bounding_box": (int(x), int(y), int(w), int(h)),
            "circularity": float(circularity),
        }
    )

lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
lbp_norm = (lbp - lbp.min()) / (lbp.max() - lbp.min() + 1e-9)
lbp_img = (lbp_norm * 255).astype("uint8")
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_lbp.png"), lbp_img)
lbp_bins = np.arange(0, lbp.max() + 2)
lbp_hist, _ = np.histogram(lbp.ravel(), bins=lbp_bins, density=True)

distances = [1, 2, 3]
angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]
glcm = greycomatrix(gray, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
glcm_features = {}
for prop in ["contrast", "dissimilarity", "homogeneity", "ASM", "energy", "correlation"]:
    glcm_features[prop] = float(greycoprops(glcm, prop).mean())

hog_vector, hog_vis = hog(
    gray,
    orientations=9,
    pixels_per_cell=(16, 16),
    cells_per_block=(2, 2),
    visualize=True,
    block_norm="L2-Hys",
    feature_vector=True,
)
hog_vis_norm = (hog_vis - hog_vis.min()) / (hog_vis.max() - hog_vis.min() + 1e-9)
hog_img = (hog_vis_norm * 255).astype("uint8")
cv2.imwrite(str(OUTPUT_DIR / f"{image_label}_hog.png"), hog_img)

print(f"Contornos detectados: {len(contours)}")
if shape_features:
    largest = max(shape_features, key=lambda feat: feat["area"])
    print(
        "Maior contorno - área: {area:.2f}, perímetro: {perimeter:.2f}, circularidade: {circularity:.3f}".format(
            **largest
        )
    )
    print(f"Bounding box (x, y, w, h): {largest['bounding_box']}")
else:
    print("Nenhum contorno detectado.")

print(f"Histograma LBP (primeiros 10 bins): {lbp_hist[:10]}")
print(f"GLCM (média das propriedades): {glcm_features}")
print(f"HOG - tamanho do vetor de características: {hog_vector.size}")


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

axes[0, 0].imshow(cv2.cvtColor(input_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title("Original")
axes[0, 0].axis("off")

axes[0, 1].imshow(gray, cmap="gray")
axes[0, 1].set_title("Grayscale")
axes[0, 1].axis("off")

axes[0, 2].imshow(gaussian, cmap="gray")
axes[0, 2].set_title("Gaussian blur")
axes[0, 2].axis("off")

axes[1, 0].imshow(median, cmap="gray")
axes[1, 0].set_title("Median blur")
axes[1, 0].axis("off")

axes[1, 1].imshow(otsu_mask, cmap="gray")
axes[1, 1].set_title("Otsu mask")
axes[1, 1].axis("off")

axes[1, 2].imshow(edges, cmap="gray")
axes[1, 2].set_title("Canny edges")
axes[1, 2].axis("off")

plt.tight_layout()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(contour_overlay, cv2.COLOR_BGR2RGB))
axes[0].set_title("Contornos")
axes[0].axis("off")

axes[1].imshow(lbp_img, cmap="gray")
axes[1].set_title("LBP")
axes[1].axis("off")

axes[2].imshow(hog_img, cmap="inferno")
axes[2].set_title("HOG")
axes[2].axis("off")

plt.tight_layout()
